---
# Crop, Resize, and Seperate Classes

- Reads original image + bbox
- Crops to bbox
- resizes to 
---

In [24]:
import os
import imutils
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

**Create Preprecessor for images**

In [6]:
# Pyimage search DL4CV, "Practioners Bundle", chapter 2
class AspectAwarePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        # store the target image width, height, and interpolation
        # method used when resizing
        self.width = width
        self.height = height
        self.inter = inter
        
    def pad_to_square(self,image):
        (w,h) = image.shape[:2]
        w_pad = (h-w)// 2 if h > w else 0
        h_pad = (w-h)// 2 if w > h else 0
        imagePad = cv2.copyMakeBorder(image,w_pad,w_pad,h_pad,h_pad,cv2.BORDER_CONSTANT,value=[0,0,0])
        #print(f"Image height={h}\nImage widt={w}")
        #print(f"This is crop width={w_pad}\nThis is crop height={h_pad}")
        return imagePad
    
    def crop_to_square(self,image,bbox):
        w,h = image.shape[:2]
        wBB,hBB = bbox[2]-bbox[0],bbox[3]-bbox[1]
        dW = (hBB-wBB) // 2 if hBB > wBB else 0
        dH = (wBB-hBB) // 2 if wBB > hBB else 0
        cropBB = [max(bbox[0]-dW,0),
                  max(bbox[1]-dH,0),
                  min(bbox[2]+dW,w),
                  min(bbox[3]+dH,h)]
        imgCrop = image[cropBB[1]:cropBB[3],cropBB[0]:cropBB[2],:]
        return imgCrop
        

    def preprocess(self, image,bbox=None,pad_to_square=False,crop_to_square=False):
        if bbox is not None:
            assert type(bbox) is tuple or \
                   type(bbox) is list  and \
                   len(bbox) == 4, \
                    "Invalid object supplied for bbox. Should be 4-tpl"
        if pad_to_square: image = self.pad_to_square(image)
        if crop_to_square: image = self.crop_to_square(image,bbox)
        # grab the dimensions of the image and then initialize
        # the deltas to use when cropping
        (h, w) = image.shape[:2]
        dW = 0
        dH = 0

        # if the width is smaller than the height, then resize
        # along the width (i.e., the smaller dimension) and then
        # update the deltas to crop the height to the desired
        # dimension
        if w < h:
            image = imutils.resize(image, width=self.width,
                inter=self.inter)
            dH = int((image.shape[0] - self.height) / 2.0)

        # otherwise, the height is smaller than the width so
        # resize along the height and then update the deltas
        # crop along the width
        else:
            image = imutils.resize(image, height=self.height,
                inter=self.inter)
            dW = int((image.shape[1] - self.width) / 2.0)

        # now that our images have been resized, we need to
        # re-grab the width and height, followed by performing
        # the crop
        (h, w) = image.shape[:2]
        image = image[dH:h - dH, dW:w - dW]

        # finally, resize the image to the provided spatial
        # dimensions to ensure our output image is always a fixed
        # size
        return cv2.resize(image, (self.width, self.height),
            interpolation=self.inter)

---

Build data structures

In [7]:
with open('../data/Anno/list_eval_partition.txt','r') as fp:
    trainPartition = [ l.rstrip('\n') for l in fp ][2:]
    trainPartition = [ l.split()[0] for l in trainPartition \
                      if l.split()[1] == 'train' or \
                         l.split()[1] == 'test']
                      #if l.split()[1] == 'val']

In [8]:
with open('../data/Anno/list_category_cloth.txt','r') as fp:
    lines = [ l.rstrip().replace('1','upper') \
                        .replace('2','lower') \
                        .replace('3','full').split() for l in fp][2:]  
    mainCatagories = { l[0]:l[1] for l in lines}

In [9]:
with open('../data/Anno/list_bbox.txt','r') as fp:
    lines = [l.rstrip('\n').split() for l in fp][2:]
    bbox_dict = { l[0]: [int(l[1]),int(l[2]),int(l[3]),int(l[4]) ] for l in lines}

Create directory structures

In [21]:
sourceDir = '../data/Img/'
targetDir = '../dataCropSQ/Img/'
targetAnnoDir = '../dataCropSQ/Anno/'

In [19]:
# create directories 
#dirSet = { l.split('/')[1] for l in bbox_dict.keys()}
#for d in dirSet:
#    targetPath = os.path.join(targetDir,d)
#    if not os.path.exists(targetPath): os.mkdir(targetPath)
#    #break



In [25]:
# create folders with all files
pp = AspectAwarePreprocessor(200,200)
trainImgList = []

In [29]:
for sImgKey in trainPartition:
    sourceImgPath = sImgKey.replace('img/',sourceDir)
    targetImgPath = sImgKey.replace('img/',targetDir)
    tDir = os.path.dirname(targetImgPath)
    if not os.path.exists(tDir): os.mkdir(tDir)
    sImg = plt.imread(sourceImgPath)
    tImg = pp.preprocess(sImg,bbox=bbox_dict[sImgKey],crop_to_square=True)
    try: 
        plt.imsave(targetImgPath,tImg)
    except:
        print(f"failed on img={sImgKey}, skipping..")
        continue
    trainImgList.append(sImgKey)  # only save image if was success
    
    break

In [ ]:
for cl_main in ['upper','lower','full']:
    with open(os.path.join(targetAnnoDir,cl_main + '_train_list.csv','w') as fp:
        fp.write('x_col, y_col\n')
        for imgk in trainImgList:
            cl_class = imgk.split('/')[1].split('_')[-1]
            if not mainCatagories[cl_class] == cl_main: continue
            arg1 = imgk.replace('img/',targetDir)
            arg2 = cl_class
            fp.write(arg1 + ',' + arg2 + ','+ arg3 + '\n')